In [1]:
import re
import json

import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from dhlab.text import nbtokenizer as tokenizer

In [2]:
with open("Andhrimner_6.xml", 'r') as f:
    xml = bs(f.read(), "lxml")

In [3]:
divs = xml.find_all("div")[1:]

In [36]:
dist = {}
texts = []

for idx,div in enumerate(divs):
    
    hw = div.find_all("signed") # evt med , {"rend": "handwritten"}
    if hw:
        for h in hw:
            h.decompose()
    
    author = div.get('resp', False)
    genre = div.get('type', 'no_type')
    
    
    if genre == 'prose':
        if author:
            with open("NINA/"+author+'_'+str(idx+1)+'.txt', 'w') as out:
                text = ' '.join(tokenizer.tokenize(re.sub('\n', ' ', re.sub('-\n', '', div.text)).strip())).strip()
                texts.append(text)
                out.write(text)
                
                if author in dist:
                    dist[author] += 1
                else:
                    dist[author] = 1
                

In [16]:
#plt.bar(*zip(*dist.items()))
#plt.show()

# Struktur:

- Hele Andhrimner som en json-struktur: {tekst: [token_1, token_n], annotasjon: {trykt:"", bh:"", asterisk:"", sjanger:"", his:""}}

In [50]:
andh = []

for div in divs:
    text = tokenizer.tokenize(re.sub('\n', ' ', re.sub('-\n', '', div.text)).strip())
        
    genre = div.get('type', 'no_type')
    his = div.get('resp', 'no_resp')
    signed = div.find_all('signed')
    
    andh.append({"text": text, "annotation": {"signed": [(s.text, s.get('rend', 'trykt')) for s in signed], "genre": genre, "his": his}})

In [51]:
with open("Andhrimner.json", 'w') as out:
    json.dump(andh, out)